In [6]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)
#output display setting
torch.set_grad_enabled(True)

In [7]:
print(torch.__version__)
print(torchvision.__version__)

1.1.0
0.3.0


In [2]:
class Network(nn.Module):
#有继承的类需要把父类写在括号里
    def __init__(self):
        super(Network, self).__init__()
#规定操作，第三行要写父类的构造器，格式为super(子类，self).__init__（）括号内可
#能有父亲类的构造参数。
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
#in_channels=1因为输入的图片为灰阶图片，此超参数由数据影响
#类的属性可以是其他类
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2= nn.Linear(in_features=120, out_features=60)
        
        self.out = nn.Linear(in_features=60, out_features=10)
#out_features=10由数据决定，分成十类。

#一般来说一个层的输出数目就是下一个层的输入数目。


        
#Network类包含属性：conv1，conv2，fc1，fc2和out一共5个属性，每个属性的值由nn
#下面的一个类定义而成（类的属性可以是其他类），类位置在nn/modole/conv中并且加
#入了适当的构造参数。其构造参数即为神经网络的“超参数”。超参数由设计者根据经验给出。
#
#Con2d为图片卷积层：包含三个参数：kernel_size，in_channels，out_features。
#kernel_size：定义了卷积核的大小
#in_channels：定义了输入的通道数
#out_channels：定义了卷积核的数量，每个卷积核都能生成一个卷积图片（feature_maps）
# out_features：全连阶层的输出，设计者决定。
#一般来说附加的卷积层会提升输出的通道，线性层会一层一层收缩。
#

    def forward(self, t): 
        #(1) input layer
        t=t
        
        #(2) hidden conv layer
        t=self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        #relu和pooling不包含权重参数，只是操作，所以直接从torch.nn.functional引用
        #大小为2的池化层会将每一个2x2的区域找出最大值并且返回，输出图片是之前的一半
        
        #(3) hidden conv layer
        t=self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        #(4) hidden linear layer 
        t = t.reshape(-1,12*4*4)
        t = self.fc1(t)
        t = F.relu(t)
        
        #(5) hidden linear layer 
        t = self.fc2(t)
        t = F.relu(t)
        
        #(6) output layer
        t = self.out(t)
        #t = F.softmax(t, dim=1)
        
        #hidden layers多用ReLU激活函数，单一预测的输出层多用softmax激活函数
        
        return t

In [10]:
train_set = torchvision.datasets.FashionMNIST(
        root='./data/FashionMNIST',
        train=True,
        download=True,
        transform=transforms.Compose([transforms.ToTensor()])
        )
train_loader = torch.utils.data.DataLoader(train_set, batch_size=10)


In [3]:
net = Network()
net

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)